Imports de libraries

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import pandas as pd
import numpy as np
import joblib
import math

jornada = 9

Definição de Constantes

In [ ]:
MAIN_PATH = '/content/drive/MyDrive/'



Invocar notebooks de funções e de constantes

In [ ]:
functions_path = MAIN_PATH + 'Colab Notebooks/Bet_Functions.ipynb'

%run '{functions_path}'


Extração do ficheiro clean

In [ ]:

df=pd.read_csv(MAIN_PATH + 'Data_Football/Liga_Portugal/Raw/Liga_Portugal_2023_24.csv')
df_clean=df[["HomeTeam",	"AwayTeam",	"FTHG",	"FTAG",	"FTR",	"HTHG",	"HTAG",	"HTR",	"HS",	"AS",	"HST",	"AST",	"HF",	"AF",	"HC",	"AC",'BWH','BWD','BWA']].copy()


Limpar nomes dos clubes -Upper Case e limpar espaços

In [ ]:
df_clean.loc[:, ['HomeTeam']] = df_clean["HomeTeam"].apply(str.upper).apply(lambda x: x.replace(' ', ''))
df_clean.loc[:, ['AwayTeam']] = df_clean["AwayTeam"].apply(str.upper).apply(lambda x: x.replace(' ', ''))

Copiar ficheiro final para a directoria clean

In [ ]:
df_clean.to_csv(MAIN_PATH + 'Data_Football/Liga_Portugal/Clean/Liga_Portugal_2023_24_clean.csv')
df_clean.head(5)

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF,HC,AC,BWH,BWD,BWA
0,SPBRAGA,FAMALICAO,1.0,2.0,A,1.0,0.0,H,13.0,12.0,2.0,4.0,17.0,12.0,3.0,4.0,1.44,4.50,6.50
1,GILVICENTE,PORTIMONENSE,5.0,0.0,H,2.0,0.0,H,8.0,9.0,5.0,3.0,11.0,14.0,1.0,1.0,2.00,3.30,3.60
2,FARENSE,CASAPIA,0.0,3.0,A,0.0,1.0,A,12.0,14.0,5.0,5.0,13.0,13.0,5.0,6.0,2.45,3.00,2.95
3,SPLISBON,VIZELA,3.0,2.0,H,2.0,0.0,H,19.0,10.0,6.0,4.0,16.0,9.0,8.0,3.0,1.18,6.75,16.00
4,RIOAVE,CHAVES,2.0,0.0,H,0.0,0.0,D,21.0,6.0,8.0,1.0,12.0,20.0,14.0,3.0,2.10,3.25,3.50


In [ ]:
dl_lp_fix, dl_lp_match_raw= split_data(jornada -1 ,df_clean)

dl_lp_match = dl_lp_match_raw.copy()
dl_lp_match.loc[:, ['HomeTeam']] = dl_lp_match["HomeTeam"].apply(str.upper).apply(lambda x: x.replace(' ', ''))
dl_lp_match.loc[:, ['AwayTeam']] = dl_lp_match["AwayTeam"].apply(str.upper).apply(lambda x: x.replace(' ', ''))
dl_lp_match

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF,HC,AC,BWH,BWD,BWA
72,AROUCA,MOREIRENSE,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.40,3.2,2.87
73,PORTIMONENSE,ESTORIL,NaN,NaN,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.45,3.4,2.70
74,BENFICA,CASAPIA,NaN,NaN,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.20,6.0,11.50
75,GUIMARAES,CHAVES,NaN,NaN,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.62,3.9,5.00
76,GILVICENTE,SPBRAGA,NaN,NaN,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.80,3.9,1.80
77,RIOAVE,FARENSE,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.35,3.3,2.87
78,ESTRELA,FAMALICAO,NaN,NaN,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.45,3.2,2.85
79,VIZELA,PORTO,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.00,4.4,1.42
80,BOAVISTA,SPLISBON,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.00,4.4,1.42


In [ ]:
all_teams_home = dl_lp_fix.HomeTeam.unique()
all_teams_away = dl_lp_fix.AwayTeam.unique()

all_teams_raw = np.concatenate([all_teams_home, all_teams_away])

all_teams = np.unique(all_teams_raw)

all_teams_stats = []

for team in all_teams:
  team_stats = calc_feats(dl_lp_fix,team)
  all_teams_stats.append(team_stats)

df_all_stats = pd.DataFrame(all_teams_stats)

Apply Model

In [ ]:
features = df_all_stats[['shots','shots_ot','shots_vs','shots_ot_vs']]
model = joblib.load('/content/drive/MyDrive/Data_Football/Liga_Portugal/Model/lp_model.joblib')

df_all_stats['exp_points'] = model.predict(features)

df_all_stats  = df_all_stats.sort_values(['exp_points'], ascending=[True]).reset_index()

df_all_stats['exp_position'] = df_all_stats_position.index.to_numpy()

df_all_stats


,index,team,shots,shots_ot,shots_vs,shots_ot_vs,points,exp_points,exp_position
0,2,BOAVISTA,75.0,35.0,111.0,49.0,12,3.646303,0
1,14,RIOAVE,76.0,26.0,96.0,42.0,5,3.957281,1
2,6,ESTRELA,83.0,24.0,115.0,44.0,9,4.988979,2
3,3,CASAPIA,84.0,27.0,102.0,43.0,8,5.052057,3
4,12,PORTIMONENSE,70.0,27.0,104.0,37.0,8,5.649275,4
5,11,MOREIRENSE,84.0,25.0,95.0,33.0,11,7.788364,5
6,9,GILVICENTE,84.0,41.0,108.0,41.0,10,8.112891,6
7,8,FARENSE,98.0,33.0,152.0,50.0,8,8.256733,7
8,5,ESTORIL,97.0,41.0,118.0,46.0,5,8.507935,8
9,0,AROUCA,97.0,31.0,76.0,34.0,5,8.582477,9


Value Betting
Prever resultados da jornada

In [ ]:
dl_lp_match[['HomeTeam', 'AwayTeam','BWH','BWD','BWA','FTR']]
dl_lp_match

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF,HC,AC,BWH,BWD,BWA
72,AROUCA,MOREIRENSE,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.40,3.2,2.87
73,PORTIMONENSE,ESTORIL,NaN,NaN,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.45,3.4,2.70
74,BENFICA,CASAPIA,NaN,NaN,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.20,6.0,11.50
75,GUIMARAES,CHAVES,NaN,NaN,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.62,3.9,5.00
76,GILVICENTE,SPBRAGA,NaN,NaN,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.80,3.9,1.80
77,RIOAVE,FARENSE,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.35,3.3,2.87
78,ESTRELA,FAMALICAO,NaN,NaN,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.45,3.2,2.85
79,VIZELA,PORTO,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.00,4.4,1.42
80,BOAVISTA,SPLISBON,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.00,4.4,1.42


In [ ]:
predictions = []

for ind,row in dl_lp_match.iterrows():

  home_team = row['HomeTeam']
  away_team = row['AwayTeam']

  home_exp_position = df_all_stats[
      df_all_stats['team']==home_team]['exp_position'].values[0]
  away_exp_position = df_all_stats[
      df_all_stats['team']==away_team]['exp_position'].values[0]


  exp_position_diff_teams = (home_exp_position - away_exp_position)

  if exp_position_diff_teams > 1:
    result = 'H'
    odds = row['BWH']
  elif abs(exp_position_diff_teams) < 1:
     result = 'D'
     odds = row['BWD']
  else:
    result = 'A'
    odds = row['BWA']

  pred = {"HomeTeam" : home_team, "AwayTeam": away_team, "Prediction": result, "Odds":odds}

  predictions.append(pred)

Build slip

In [ ]:
df_slip = pd.DataFrame(predictions)

df_slip = dl_lp_match.merge(df_slip, on = ['HomeTeam', 'AwayTeam'])[['HomeTeam','AwayTeam','Prediction','Odds','FTR']]


df_slip['Win'] = np.where(
     df_slip['Prediction']==df_slip['FTR']
    , df_slip['Odds']-1
    ,-1
)

df_slip

,HomeTeam,AwayTeam,Prediction,Odds,FTR,Win
0,AROUCA,MOREIRENSE,H,2.40,A,-1.00
1,PORTIMONENSE,ESTORIL,A,2.70,H,-1.00
2,BENFICA,CASAPIA,H,1.20,D,-1.00
3,GUIMARAES,CHAVES,A,5.00,H,-1.00
4,GILVICENTE,SPBRAGA,A,1.80,D,-1.00
5,RIOAVE,FARENSE,A,2.87,A,1.87
6,ESTRELA,FAMALICAO,A,2.85,H,-1.00
7,VIZELA,PORTO,A,1.42,A,0.42
8,BOAVISTA,SPLISBON,A,1.42,N,-1.00


In [ ]:
df_slip.to_csv(MAIN_PATH + 'Data_Football/Liga_Portugal/Predicts/slip_jornada_' + ' ' + str(jornada)  + '.csv')

Calc sum and append to sumary

In [ ]:
gameweek_winning = df_slip['Win'].sum()


game_week_summary = [{'Game Week': jornada,
            'Game Week Winning': gameweek_winning}]
df_game_week_summary = pd.DataFrame(game_week_summary)

df_game_week_summary.to_csv(MAIN_PATH + 'Data_Football/Liga_Portugal/Predicts/Liga_Portugal_2023_24_Sumary.csv', mode='a', header=False, index=False)

df_game_week_summary = pd.read_csv(MAIN_PATH + 'Data_Football/Liga_Portugal/Predicts/Liga_Portugal_2023_24_Sumary.csv')

df_game_week_summary

,2,-2.45
0,3,-7.00
1,4,0.95
2,5,-1.91
3,6,6.19
4,7,-5.86
5,8,2.71
6,9,-1.23
7,9,-4.71
8,9,-4.71
